In [1]:
!pip install torch_geometric
!pip install --upgrade keras -qq
!git clone https://github.com/anas-rz/k3-node.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.4 which is incompatible.
Cloning into 'k3-node'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 489 (delta 40), reused 66 (delta 26), pack-reused 401
Receiving objects: 100% (489/489), 355.01 KiB | 1.78 MiB/s, done.
Resolving deltas: 100% (276/276), done.


In [1]:
import os, sys
sys.path.append('./k3-node')
os.environ['KERAS_BACKEND'] = 'torch'

In [2]:
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

import keras
from keras import layers, Model, ops


from k3_node.layers import GatedGraphConv
from k3_node.utils import edge_index_to_adjacency_matrix

dataset = 'Cora'
path = osp.join(osp.dirname(osp.realpath('.')), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [3]:
class Net(Model):
    def __init__(self):
        super().__init__()
        self.dropout1 = layers.Dropout(0.3)
        self.dropout2 = layers.Dropout(0.3)
        self.lin1 = layers.Dense(16)
        self.prop1 = GatedGraphConv(128, 3)
        self.prop2 = GatedGraphConv(128, 2)
        self.lin2 = layers.Dense(dataset.num_classes)

    def call(self, data=None):
        x = data.x
        adj = edge_index_to_adjacency_matrix(data.edge_index)
        x = self.dropout1(x)
        x = ops.relu(self.lin1(x))

        x = self.prop1((x, adj))
        x = self.prop2((x, adj))
        x = self.dropout2(x)
        x = self.lin2(x)
        return ops.log_softmax(x, axis=1)

In [4]:
@torch.no_grad()
def test(model):
    model.eval()
    out, accs = model(data=data), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [5]:
model = Net()
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
best_val_acc = 0
for epoch in range(1, 51):
    # Forward pass
    out = model(data=data)
    loss = loss_fn(data.y[data.train_mask], out[data.train_mask])

    # Backward pass
    model.zero_grad()
    trainable_weights = [v for v in model.trainable_weights]

    # Call torch.Tensor.backward() on the loss to compute gradients
    # for the weights.
    loss.backward()
    gradients = [v.value.grad for v in trainable_weights]

    # Update weights
    with torch.no_grad():
        optimizer.apply(gradients, trainable_weights)

    train_acc, val_acc, tmp_test_acc = test(model)

    print(
        f"Training loss at epoch {epoch}: {loss.detach().numpy():.4f}"
    )
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Training loss at epoch 1: 1.9491
Epoch: 001, Train: 0.1571, Val: 0.1620, Test: 0.1450
Training loss at epoch 2: 1.8746
Epoch: 002, Train: 0.2714, Val: 0.1840, Test: 0.1890
Training loss at epoch 3: 1.8219
Epoch: 003, Train: 0.2357, Val: 0.1840, Test: 0.1890
Training loss at epoch 4: 1.7723
Epoch: 004, Train: 0.3357, Val: 0.2980, Test: 0.3100
Training loss at epoch 5: 1.7175
Epoch: 005, Train: 0.3500, Val: 0.3360, Test: 0.3440
Training loss at epoch 6: 1.6714
Epoch: 006, Train: 0.3500, Val: 0.3360, Test: 0.3440
Training loss at epoch 7: 1.6145
Epoch: 007, Train: 0.3643, Val: 0.3360, Test: 0.3440
Training loss at epoch 8: 1.5644
Epoch: 008, Train: 0.3857, Val: 0.3660, Test: 0.3690
Training loss at epoch 9: 1.5077
Epoch: 009, Train: 0.4214, Val: 0.4300, Test: 0.4280
Training loss at epoch 10: 1.4584
Epoch: 010, Train: 0.4071, Val: 0.4380, Test: 0.4300
Training loss at epoch 11: 1.4119
Epoch: 011, Train: 0.4357, Val: 0.4520, Test: 0.4390
Training loss at epoch 12: 1.3715
Epoch: 012, Train: